In [1]:
import os
import json
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Best Global Universities

**Особенности данного рейтинга**, что на сайте не сохраняется ретроспектива. Парсить будем что есть (на момент написания рейтинг 2022-2023).

Для начала станем немного больше похожими на браузер.

In [12]:
session = requests.session()
session.headers.update({
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ru',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
})

## Общий рейтинг

In [3]:
data_list = []
url_tpl = 'https://www.usnews.com/education/best-global-universities/search?format=json&page='
url_first = url_tpl + '1'
resp_first = session.get(url_first)

if resp_first.status_code == 200:
    data_first = resp_first.json()
    total_pages = data_first['total_pages']
    data_list += data_first['items']

    for page_number in range(2, total_pages  + 1):
        url = url_tpl + str(page_number)
        resp = session.get(url)

        if resp.status_code == 200:
            json_data = resp.json()
            data_list += json_data['items']
        else:
            print(f"{url} return status code {resp.status_code}")

        # Бережём источник данных
        time.sleep(1)
else:
    print(f"{url_first} return status code {resp_first.status_code}")

ranking = pd.DataFrame(data_list)
ranking.head()

,url,id,name,city,country_name,three_digit_country_code,ranks,stats,image_url,blurb
0,https://www.usnews.com/education/best-global-u...,166027,Harvard University,Cambridge (U.S.),United States,USA,"[{'value': '1', 'is_tied': False, 'is_ranked':...","[{'value': '100.0', 'label': 'Global Score'}, ...",{'src': 'https://www.usnews.com/object/image/0...,"Founded in 1636, Harvard University is the old..."
1,https://www.usnews.com/education/best-global-u...,166683,Massachusetts Institute of Technology (MIT),Cambridge (U.S.),United States,USA,"[{'value': '2', 'is_tied': False, 'is_ranked':...","[{'value': '97.7', 'label': 'Global Score'}, {...",{'src': 'https://www.usnews.com/object/image/0...,"Massachusetts Institute of Technology, founded..."
2,https://www.usnews.com/education/best-global-u...,243744,Stanford University,Stanford,United States,USA,"[{'value': '3', 'is_tied': False, 'is_ranked':...","[{'value': '95.2', 'label': 'Global Score'}, {...",{'src': 'https://www.usnews.com/object/image/0...,Stanford University was founded in 1885 and is...
3,https://www.usnews.com/education/best-global-u...,110635,University of California Berkeley,Berkeley,United States,USA,"[{'value': '4', 'is_tied': False, 'is_ranked':...","[{'value': '88.7', 'label': 'Global Score'}, {...",{'src': 'https://www.usnews.com/object/image/0...,The University of California—Berkeley is situa...
4,https://www.usnews.com/education/best-global-u...,503637,University of Oxford,Oxford,United Kingdom,GBR,"[{'value': '5', 'is_tied': False, 'is_ranked':...","[{'value': '86.8', 'label': 'Global Score'}, {...",{'src': 'https://www.usnews.com/object/image/0...,The exact date of the University of Oxford’s f...


Вытащим данные про рейтинг из столбца `ranks` и сохраним результаты, а далее выстроим, для примера, производный рейтинг стран по медианной позиции университетов в рейтинге (посмотрим ТОП 10).

In [4]:
for i, r in ranking.iterrows():
    rank_value = r['ranks'][0]['value']

    ranking.loc[i, 'ranks.value'] = np.NaN if rank_value == 'Unranked' else int(rank_value.replace(',', ''))
    ranking.loc[i, 'ranks.is_tied'] = r['ranks'][0]['is_tied']
    ranking.loc[i, 'ranks.is_ranked'] = r['ranks'][0]['is_ranked']
    ranking.loc[i, 'ranks.label'] = r['ranks'][0]['label']

# Сохраняем результаты
ranking.to_csv('data/best_global_universities.csv', index=False)
ranking.to_excel('data/best_global_universities.xlsx', index=False)

# ТОП 10 стран по медиане
ranking[['country_name', 'id', 'ranks.value']].groupby(by='country_name').agg({
    'id': 'count',
    'ranks.value': 'median'
}).rename(columns={
    'id': 'count',
    'ranks.value': 'rank_median'
}).sort_values(by='rank_median').head(10)

,count,rank_median
country_name,,
Netherlands,15,97.5
Hong Kong,7,100.0
Switzerland,12,150.0
Singapore,4,231.0
Denmark,7,261.0
Australia,39,273.0
Belgium,11,292.0
Sweden,19,346.5
Finland,11,447.0


## Предметные

По предметам данных получается намного больше, поэтому разобьём весь процесс на этапы. Начнём со списка предметов (тематик).

In [5]:
subjects = []
subject_first_url = 'https://www.usnews.com/education/best-global-universities/rankings'
subject_first_resp = session.get(subject_first_url)

if subject_first_resp.status_code == 200:
    subject_html = subject_first_resp.text
    subject_soup = BeautifulSoup(subject_html)
    subject_select = subject_soup.find('select', {'name': 'subject'})

    for o in subject_select.find_all('option'):
        if o['value']:
            subjects.append({
                'title': o.text,
                'value': o['value']
            })
else:
    print(f"{subject_first_url} return status code {subject_first_resp.status_code}")

pd.DataFrame(subjects).head()

,title,value
0,Agricultural Sciences,agricultural-sciences
1,Artificial Intelligence,artificial-intelligence
2,Arts and Humanities,arts-and-humanities
3,Biology and Biochemistry,biology-biochemistry
4,Biotechnology and Applied Microbiology,biotechnology-applied-microbiology


Теперь скачаем данные по тематикам и сохраним их на диске. Это займёт какое-то время, наберитесь терпения. Процесс можно отслеживать в папке, куда сохраняются данные `data/downloads/bgu_by_subjects`.

Если что-то «залипнет», останавливайте и перезапускайте процесс, в том числе, на всякий случай, блок выше, где создаётся объект `session`. Код каждый запуск будет скачивать только то, что не успел скачать в прошлый раз.

In [13]:
subject_data_folder = 'data/downloads/bgu_by_subjects'
subject_url_prefix = 'https://www.usnews.com/education/best-global-universities'

for i, s in enumerate(subjects):
    subject_value = s['value']
    subject_dir = f"{subject_data_folder}/{subject_value.replace('-', '_')}"

    # Создаём папки для данных, если их не существует
    if not os.path.exists(subject_dir):
        os.makedirs(subject_dir)

    url_subject_tpl = f"{subject_url_prefix}/{subject_value}?format=json&page="
    first_url_subject = url_subject_tpl + '1'
    first_resp_subject = session.get(first_url_subject)

    if first_resp_subject.status_code == 200:
        first_json_subject = first_resp_subject.json()
        total_pages_subject = first_json_subject['total_pages']

        with open(f"{subject_dir}/1.json", 'w') as f:
            f.write(json.dumps(first_resp_subject.json()['items'], ensure_ascii=False, indent=4))

        for page_number in range(2, total_pages_subject + 1):
            subject_page_file_data = f"{subject_dir}/{page_number}.json"

            if os.path.isfile(subject_page_file_data):
                continue

            url_subject = url_subject_tpl + str(page_number)
            resp_subject = session.get(url_subject)

            if resp_subject.status_code == 200:
                with open(subject_page_file_data, 'w') as f:
                    f.write(json.dumps(resp_subject.json()['items'], ensure_ascii=False, indent=4))
            else:
                print(f"{url_subject} return status code {resp_subject.status_code}")

            # Бережём источник данных
            time.sleep(1)
    else:
        print(f"{first_url_subject} return status code {first_resp_subject.status_code}")

Соберём всё в кучку из скачанных файлов.

In [14]:
df_by_subject = pd.DataFrame()

for s in subjects:
    subject_value = s['value']
    subject_dir = f"{subject_data_folder}/{subject_value.replace('-', '_')}"
    bgu_files_numbers_in_subject = list(sorted([int(x.replace('.json', '')) for x in os.listdir(subject_dir)]))

    for n in bgu_files_numbers_in_subject:
        file_path = f"{subject_dir}/{n}.json"
        json_data = json.load(open(file_path, 'r'))
        chunk = pd.json_normalize(json_data)

        for i, r in chunk.iterrows():
            for rank_idx, rank in enumerate(r['ranks']):
                rank_value = rank['value']
                chunk.loc[i, f'rank-{rank_idx + 1}.value'] = np.NaN if rank_value == 'Unranked' else int(rank_value.replace(',', ''))
                chunk.loc[i, f'rank-{rank_idx + 1}.is_tied'] = rank['is_tied']
                chunk.loc[i, f'rank-{rank_idx + 1}.is_ranked'] = rank['is_ranked']
                chunk.loc[i, f'rank-{rank_idx + 1}.label'] = rank['label']

        df_by_subject = pd.concat([df_by_subject, chunk], ignore_index=True)

df_by_subject.to_csv('data/best_global_universities_subject.csv', index=False)
df_by_subject.to_excel('data/best_global_universities_subject.xlsx', index=False)
df_by_subject.head()

,url,id,name,city,country_name,three_digit_country_code,ranks,stats,blurb,image_url.src,image_url.medium,image_url.large,rank-1.value,rank-1.is_tied,rank-1.is_ranked,rank-1.label,rank-2.value,rank-2.is_tied,rank-2.is_ranked,rank-2.label
0,https://www.usnews.com/education/best-global-u...,501112,Wageningen University & Research,Wageningen,Netherlands,NLD,"[{'value': '1', 'is_tied': False, 'is_ranked':...","[{'value': '100.0', 'label': 'Subject Score'},...",,NaN,NaN,NaN,1.0,False,True,Best Universities for Agricultural Sciences,89.0,True,True,Best Global Universities
1,https://www.usnews.com/education/best-global-u...,500688,China Agricultural University,Beijing,China,CHN,"[{'value': '2', 'is_tied': False, 'is_ranked':...","[{'value': '96.3', 'label': 'Subject Score'}, ...",,NaN,NaN,NaN,2.0,False,True,Best Universities for Agricultural Sciences,332.0,False,True,Best Global Universities
2,https://www.usnews.com/education/best-global-u...,504800,Jiangnan University,Wuxi,China,CHN,"[{'value': '3', 'is_tied': False, 'is_ranked':...","[{'value': '93.9', 'label': 'Subject Score'}, ...",,NaN,NaN,NaN,3.0,False,True,Best Universities for Agricultural Sciences,598.0,True,True,Best Global Universities
3,https://www.usnews.com/education/best-global-u...,505115,South China University of Technology,Guangzhou,China,CHN,"[{'value': '4', 'is_tied': False, 'is_ranked':...","[{'value': '91.7', 'label': 'Subject Score'}, ...",,NaN,NaN,NaN,4.0,False,True,Best Universities for Agricultural Sciences,219.0,True,True,Best Global Universities
4,https://www.usnews.com/education/best-global-u...,166629,University of Massachusetts Amherst,Amherst,United States,USA,"[{'value': '5', 'is_tied': False, 'is_ranked':...","[{'value': '88.3', 'label': 'Subject Score'}, ...",,NaN,NaN,NaN,5.0,False,True,Best Universities for Agricultural Sciences,160.0,True,True,Best Global Universities


Давайте посмотрим ТОП 10 предметных рейтингов по количеству университетов, в которых представлена 🇷🇺 Россия.

In [15]:
df_by_subject[df_by_subject['three_digit_country_code'] == 'RUS'][['rank-1.label', 'id']].\
    groupby(by='rank-1.label').count().\
    rename(columns={'id': 'count'}).sort_values(by='count', ascending=False)

,count
rank-1.label,
Best Universities for Physics,20
Best Universities for Chemistry,17
Best Universities for Materials Science,14
Best Universities for Engineering,9
Best Universities for Physical Chemistry,9
Best Universities for Optics,8
Best Universities for Mathematics,7
Best Universities for Condensed Matter Physics,6
Best Universities for Geosciences,5
